In [1]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from typing import Iterable, Union
import proliferation as pro
import networkx as nx
from scipy import optimize

# Color codes
moran_color = "tab:blue"
bernoulli_color = "tab:red"
binomial_color = "tab:orange"

def prod(els :Iterable[float | int]):
    """The product of an iterable
    """
    p = 1
    for x in els:
        p *= x
    return p

def rationalize(f :float):
    """Rationalize a number
    """
    r = sp.Rational(f)
    return r.numerator, r.denominator

def rat_linspace(l :int, u :int, n :int):
    return [ (l+(u-l)*i, n-1) for i in range(n) ]


# Mayor's formulas
## $K_N$
### Absorption on $K_n$

In [2]:
# Absorption time for Moran on complete graphs
def pMp(j :int, N :int, r :float):
    # son can kill father
    return r*j / (r*j + (N-j)) * (N-j) / (N-1)

def tauM(N :int):
    def time(i: int, r :float):
        val = r**(N-i) * (r**i-1) / (r**N-1)
        val *= sum( 
            (pMp(j,N,r) * r**(N-j-1))**-1 * (r**(N-j)-1) / (r-1) 
            for j in range(1,N)
        )
        val -= sum(
            (pMp(j,N,r) * r**(i-j-1))**-1 * (r**(i-j)-1)/(r-1)
            for j in range(1,i)
        )
        return val
    
    return time

In [3]:
# Absorption time for Bernoulli proliferation on complete graphs
def tauB(N:int):
    def time(i: int, r:float, p:float):
        val = sum(
            (1 + r * p - j/(N-1))**-1 * \
            (r*(j+1) + N - j -1) / (j+1) \
            * prod( 
                (1-k/(N-1)) / (1 + r*p - k / (N-1))
                for k in range(j+1,i)
            )
            for j in range(i)
        )
        return val
    
    return time    

### Fixation on $K_n$

In [4]:
# Fixation probability for Moran in the complete graph
def PhiM(N :int):
    def probability(i :int, r :float):
        return (1-1/r**i)/(1-1/r**N)
    return probability

# Fixation probability for Bernoulli in the complete graph
def _PhiB(N :int, i :int, r :float, p :float):
    if i == 0:
        return 0
    if i == N:
        return 1
    val = r*i*p + _PhiB(N, i-1, r, p) * (N-i) * i / (N-1)
    val /= r*i*p + (N-i) * i / (N-1)
    
    return val

def PhiB(N :int):
    return lambda i, r, p : _PhiB(N, i, r, p)

def critical(N :int, r :float):
    return (r**(N-1) - r**(N-2)) / (r**(N-1) - 1)

In [5]:
# Fixation time for Moran on the complete graph
def TM(N :int):
    fp = PhiM(N)
    
    def time(i :int, r :float):
        assert i==1, "Not implemented yet!"
        
        # Δ_i = ( Φ_{i-2} Δ_{i-1} (N-i+1)(i-1) - Φ_{i-1} w_{i-1} (N-1)) / (Φ_i r (i-1) (N-i+1))
        # Δ_2 = - Φ_1 w_1 / (Φ_2 r)
        # -(Δ_2 + Δ_3 + ... + Δ_N) = T_1

        w = [ r*k + N-k for k in range(N+1) ]

        def delta(k :int):
            if k == 2:
                return -w[1] / r * fp(1, r) / fp(2, r)
            
            val = fp(k-2, r) * delta(k-1) * (N-k+1) * (k-1) 
            val -= fp(k-1, r) * w[k-1] * (N-1)
            val /= fp(k, r) * r * (k-1) * (N-k+1)
            return val
            
        return -sum(delta(k) for k in range(2, N+1) )
    
    return time

# Son iguales... la de abajo la original

#def TM(N :int):
#    fp = PhiM(N)
#    
#    def time(i :int, r :float):
#        val = 1*sum(
#                fp(j,r)/pMp(j,N,r) * \
#                sum(1/r**(k-j)
#                    #prod(1/r for l in range(j+1, k+1)) 
#                    for k in range(j, N))
#            for j in range(1,N))
#        
#        val -= 1/fp(i,r) * sum(
#                fp(j, r) / pMp(j,N,r) * \
#                sum(1/r**(k-j)
#                    #prod(1/r for l in range(j+1, k+1)) 
#                    for k in range(j,i))
#            for j in range(1,i))
#        
#        return val
#    return time

In [6]:
# Fixation time for Bernoulli proliferation on the complete graph
def TB(N : int):
    fp = PhiB(N)
    
    def time(i :int, r :float, p :float):
        val = sum(
                1/(1 + r*p - j/(N-1)) * (N-(j+1) + r*(j+1)) / (j+1) * \
                fp(j+1,r,p) / fp(i,r,p) *
                prod(
                    (1-k/(N-1)) / (1+r*p-k/(N-1)) 
                for k in range(j+1,i))
            for j in range(i))
        return val
    return time


In [4]:
tauM(3)(1, sp.var("r")).factor()

(3*r**2 + 4*r + 2)/(r**2 + r + 1)

In [10]:
tauB(3)(1, sp.var("r"), sp.var("p")).factor()

(r + 2)/(p*r + 1)

In [8]:
TM(3)(1, sp.var("r")).factor()

3*(r + 1)**2/(r**2 + r + 1)

In [11]:
TB(3)(1, sp.var("r"), sp.var("p")).factor()

(r + 2)/(p*r + 1)

In [8]:
#for N in range(3,31): 
for N in range(40,101,10):
    #N = 3
    rs = np.linspace(1, 5, 300)[1:]

    fig,axs = plt.subplots(1, 2, figsize=(10,4), sharex=True, sharey=True)
    plt.tight_layout()
    axs[0].set_ylabel("Mean time")

    for ax, name, title in zip(axs, ["tau", "T"], ['Absorption', 'Fixation']):
        fM = globals()[f"{name}M"](N)
        fB = globals()[f"{name}B"](N)

        tmp1 = np.array([fM(1,v) for v in rs])
        tmp2 = np.array([fB(1, v, critical(N,v)) for v in rs])

        ax.plot(rs , tmp1, label="Moran", c=moran_color)
        ax.plot(rs , tmp2, label="Bernoulli w/ $p_c$", c=bernoulli_color)
        #ax.plot(rs , tmp1/tmp2, "--", label="quotient", c="k")
        ax.set_xlabel("Fitness")
        ax.set_title(f"{title} time on $K_{{{N}}}$ ")
        ax.legend()
    plt.tight_layout()
    plt.savefig(f"K{N}.pdf")
    plt.yscale("log")
    plt.savefig(f"K{N}-log.pdf")
    plt.close()
    

## The cycle $C_N$
### Absorption on $C_N$

In [2]:
def critical(N :int, r :float):
    def f(p :float):
        P = sum(r**-j for j in range(1,N))
        P -= 2/(1+2*r*p) * sum( (2*r*p)**-j for j in range(0,N-1))
        #P *= (1+2*r*p) # Those quantities are always positive
        #P *= p**(N-2)
        return P
    
    return optimize.bisect(f, 0, 1, maxiter=54)

In [3]:
#def critical(N :int, r :float):
#    p = sp.var("p")
#    
#    P = sum(r**-j for j in range(1,N))
#    P -= 2/(1+2*r*p) * sum( (2*r*p)**-j for j in range(0,N-1))
#    P *= (1+2*r*p)
#    P *= p**(N-2)
#
#    P = sp.poly(P.simplify())
#    roots = [ r for r in P.nroots() if r.is_real and 0<=r<=1]
#    
#    assert len(roots), "Too many roots"
#    
#    return float(roots[0])

# Fixation probability for Moran in the complete graph... and in the cycle
def PhiM(N :int):
    def probability(i :int, r :float):
        return (1-1/r**i)/(1-1/r**N)
    return probability


def _PhiB(i,r,p,N):
    if i == N:
        return 1
    if i == 0:
        return 0
    if i == 1:
        val = 1 + sum( (2*r*p)**-k for k in range(N-1) ) / (1/2+r*p)
        return 1/val
    
    val = (sum((2*r*p)**-j for j in range(i-1)) * 2 / (1+2*r*p) + 1)* _PhiB(1, r, p, N)
            
    return val

def PhiB(N :int):
    return lambda i,r,p : _PhiB(i,r,p,N)

In [4]:
# Absorption time for the cycle in the Moran process
def tauM(N :int):
    def time(i :int, r :int):
        assert i == 1, "Not implemented yet"
        
        val = r**(N-1) * (r-1) / (r**N - 1)
        val *= sum( r**(-(i-j+1)) * (N-j + j*r) for i in range(1, N) for j in range(1, i+1) )
        
        return val
    return time

In [5]:
# Absorption time for the cycle with Bernoulli proliferation
def tauB(N :int):
    def time(i :int, r :float, p :float):
        assert i == 1, "Not implemented yet other case"
        
        val = sum(
            (1 / (2*r*p))**(i-j+1) * (N-j + j*r)
            for i in range(2, N) for j in range(2, i+1)
        )
        val += sum(1/(2*r*p)**i for i in range(0,N-1)) * N / (1 + 2*r*p)
        val /= 1 + 2/(1+2*r*p) * sum(1/(2*r*p)**i for i in range(N-1))
        
        return val
        
    return time    

In [6]:
# Fixation time for the cycle on the Moran process
def TM(N :int):
    fm = PhiM(N)
    
    def time(i :int, r: float):
        # Δ_i = (Φ_{i-2} Δ_{i-1} - Φ_{i-1} w_{i-1}) / (Φ_i r)
        # -(Δ_2 + Δ_3 + ... + Δ_N) = T_1
        # Δ_2 = -w_1 / r * Φ_1/Φ_2

        assert i == 1, "Not implemented"

        w = [ r*k + N-k for k in range(N+1) ]
        
        def delta(k :int):
            if k == 2:
                return -w[1] * fm(1, r) / fm(2, r) / r
            
            return (fm(k-2, r) * delta(k-1) - fm(k-1, r) * w[k-1]) / (fm(k, r) * r)
        
        return -sum(delta(k) for k in range(2, N+1))
    
    return time

#def TM(N :int):
#    fM = PhiM(N)
#    
#    def time(i :int, r :float):
#        assert i == 1, "Not implemented"
#        
#        val = (2*r)**(N-1) * (2*r-1)
#        val /= (2*r)**N - 1
#        val *= sum( (2*r)**(j-i-1) * (N-j+r*j) * fM(j, r) 
#                    for i in range(1, N) for j in range(1,i+1)
#                  )
#        
#        return val
#    
#    return time

In [7]:
# Fixation time on the cycle for the Bernoulli process
def TB(N :int):
    fB = PhiB(N)
    
    def time(i :int, r :float, p :float):
        val = sum( 
            (2*r*p)**(j-i-1) * (N-j+r*j) * fB(j,r,p) / fB(1,r,p)
            for i in range(2,N) for j in range(2,i+1))
        val -= ( fB(2,r,p)/fB(1,r,p)*(N-2+2*r) - 2*(N-1+r) )/(1+2*r*p) * \
            sum((2*r*p)**-i for i in range(N-1))
        
        val /= 1 + 2/(1+2*r*p) * sum((2*r*p)**-i for i in range(N-1))
        
        return val
    
    return time

In [8]:
#critical = pro.criticalProbabilityBernoulli(nx.cycle_graph(N))

In [11]:
#for N in range(3,31): 
for N in range(40,101,10):
#    N = 3

    rs = np.linspace(1, 5, 300)[1:]

    fig,axs = plt.subplots(1, 2, figsize=(10,4), sharex=True, sharey=True)
    plt.tight_layout()
    axs[0].set_ylabel("Mean time")

    for ax, name, title in zip(axs, ["tau", "T"], ['Absorption', 'Fixation']):
        fM = globals()[f"{name}M"](N)
        fB = globals()[f"{name}B"](N)

        ax.plot(rs , [fM(1,v) for v in rs], label="Moran", c=moran_color)
        ax.plot(rs , [fB(1, v, critical(N, v)) for v in rs], label="Bernoulli w/ $p_c$", c=bernoulli_color)
        ax.set_xlabel("Fitness")
        ax.set_title(f"{title} time on $C_{{{N}}}$ ")
        ax.legend()
    plt.tight_layout()
    plt.savefig(f"C{N}.pdf")
    plt.yscale("log")
    plt.savefig(f"C{N}-log.pdf")
    plt.close()

/tmp/ipykernel_22847/2338144086.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  P -= 2/(1+2*r*p) * sum( (2*r*p)**-j for j in range(0,N-1))
/tmp/ipykernel_22847/2338144086.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  P -= 2/(1+2*r*p) * sum( (2*r*p)**-j for j in range(0,N-1))
/tmp/ipykernel_22847/2338144086.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  P -= 2/(1+2*r*p) * sum( (2*r*p)**-j for j in range(0,N-1))
/tmp/ipykernel_22847/2338144086.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  P -= 2/(1+2*r*p) * sum( (2*r*p)**-j for j in range(0,N-1))
/tmp/ipykernel_22847/2338144086.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  P -= 2/(1+2*r*p) * sum( (2*r*p)**-j for j in range(0,N-1))
/tmp/ipykernel_22847/2338144086.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  P -= 2/(1+2*r*p) * sum( (2*r*p)**-j for j in range(0,N-1))
/tmp/ipykernel_22847/2338144086.py:4: Ru

## The star $K_{1,N}$
### Absorption on $K_{1,N}$

In [2]:
# Absorption time for the cycle in the Moran process
def tauM(N :int):
    def time(i :int, r :float):
        assert i == 1, "Not implemented yet"
        
        w1 = N-1 + r
        def E(j: int):
            return 1 + (N-j) / (N-1) / (r*p)**2 * (((N-1)*r*p + 1)**j - 1) / ((N-1)*r*p+1)**(j-1) 
        def D(i: int):
            return 1 + sum(
                (N-1-j) / (N-1-j+r*p) *
                prod( (N-1-k+r*p) / (((N-1)*r*p + 1)*r*p) for k in range(i,j+1) )
                for j in range(i,N-1)
            )
            
        
        val = (N-1)**2*w1/(1+(N-1)*r*p) + 1
        val += ((r*p)/(N-1+r*p) + (N-1)**2 * r*p / ((N-1)*r*p+1)) \
                        * sum( D(ii) * E(ii) for ii in range(1,N)) / D(1)
        val /= N
        
        return val
    return time

# Absorption time for the cycle with Bernoulli proliferation
def tauB(N :int):
    def time(i :int, r :float, p :float):
        assert i == 1, "Not implemented yet"
        w1 = N-1 + r
        w2 = N-2 + 2*r
        
        ret = (N-1)**2 * ( r*p*w2 + w1*(N-2+r*p) )
        ret /= (N-1) * (r*p)**2 + r*p + N-2
        ret +=  w1 / (N-1 + r*p)
        ret /= N
        
        return val
        
    return time    

In [4]:
tauM(6)(1,sp.var("r"))

NameError: name 'p' is not defined